# Appendix 1 - Transfer Learning with MONK

# Steps to take

1. Create a network
    - Creating base network
    - Visualize layers
    - Creating loss function module
    - Creating optimizer module [Set learning rates here]
    - Creating learning rate scheduler
    
    
2. Data prepraration
    - Creating a data transformer
    - Downloading and storing dataset
    - Applying transformation
    - Understanding dataset
    - Loading the transformed dataset [Set batch size and number of parallel processors here]
    
    
3. Setting up data - plotters


4. Training
    - Set Epoch
    - Train model
    
    
5. Testing
    - Just after training
    - After loading saved weights


In [1]:
!git clone https://github.com/Tessellate-Imaging/monk_v1

Cloning into 'monk_v1'...
remote: Enumerating objects: 773, done.
remote: Counting objects: 100% (773/773), done.
remote: Compressing objects: 100% (482/482), done.
remote: Total 773 (delta 349), reused 672 (delta 265), pack-reused 0
Receiving objects: 100% (773/773), 4.74 MiB | 3.68 MiB/s, done.
Resolving deltas: 100% (349/349), done.


In [ ]:
#!pip install -r monk_v1/installation/requirements_cu9.txt    
#Select Installation file as per the system

In [1]:
import sys
sys.path.append("monk_v1/monk/")
from pytorch_prototype import prototype

# Step 1 - Create experiment
ptf = prototype(verbose=1);
ptf.Prototype("exp-1", "proj-1");

Pytorch Version: 1.2.0

Experiment Details
    Project: exp-1
    Experiment: proj-1
    Dir: /home/abhi/Desktop/Work/tess_tool/gui/v0.3/finetune_models/Resources/v0.1/workspace/exp-1/proj-1/



In [ ]:
'''
original code - snippet

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'cat_dog/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
                  
                  


batch = 4
parallel_processors = 3

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch,
                                             shuffle=True, num_workers=parallel_processors)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

'''

In [2]:
# Replicating same using Monk


ptf.Dataset_Params(dataset_path=["cat_dog/train", "cat_dog/val"],
                   input_size=224, 
                   batch_size=4, shuffle_data=True, num_processors=3);

# Transform
ptf.apply_random_horizontal_flip(train=True);
ptf.apply_normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], train=True, val=True, test=True);


ptf.Dataset();

Dataset Details
    Train path:     cat_dog/train
    Val path:       cat_dog/val
    CSV train path: None
    CSV val path:  None

Dataset Params
    Input Size:  224
    Batch Size:  4
    Data Shuffle: True
    Processors:   3

Pre-Composed Train Transforms
[{'RandomHorizontalFlip': {'p': 0.5}}, {'Normalize': {'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225]}}]

Pre-Composed Val Transforms
[{'Normalize': {'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225]}}]

Dataset Numbers
    Num train images: 200
    Num val images:   50
    Num classes:      2



In [ ]:
'''
Original Code Snippet

model_ft = models.resnet18(pretrained=True) # Using pretrained models
num_ftrs = model_ft.fc.in_features # Adding a new layer to next final layer
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft.cuda()


'''

In [3]:
# Replicating same using Monk

ptf.Model_Params(model_name="resnet18", freeze_base_network=True, 
                 use_gpu=True, use_pretrained=True);
ptf.Model();

Model Params
    Model name:           resnet18
    Use Gpu:              True
    Use pretrained:       True
    Freeze base network:  True

Model Details
    Loading pretrained model
    Model Loaded on device
        Model name:           resnet18
        Num layers in model:  31
        Num trainable layers: 1



In [ ]:
'''
Original Code Snippet

cross_entropy_loss = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

'''

In [4]:
# Replicating same using Monk
ptf.optimizer_sgd(0.001);
ptf.lr_fixed();
ptf.loss_softmax_crossentropy()

Optimizer
    Name:          sgd
    Learning rate: 0.001
    Params:        {'lr': 0.001, 'dampening': 0, 'nesterov': False, 'weight_decay': 0, 'momentum': 0}

Learning rate scheduler
    Name:   fixed
    Params: {}

Loss
    Name:          softmaxcrossentropy
    Params:        {'weight': None, 'size_average': None, 'ignore_index': -100, 'reduce': None, 'reduction': 'mean'}



In [ ]:
'''
Original Code Snippet

def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()

    # A shallow copy constructs a new compound object and then (to the extent possible) inserts 
    #    references into it to the objects found in the original. 
    # A deep copy constructs a new compound object and then, recursively, inserts copies into it of 
    #    the objects found in the original.
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.cuda()
                labels = labels.cuda()

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    torch.save(model.state_dict(), "cat_dog_model")
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

model_ft = train_model(model_ft, cross_entropy_loss, optimizer_ft,
                       num_epochs=5)

'''

In [5]:
# Replicating using Monk

ptf.Training_Params(num_epochs=4, display_progress=True, display_progress_realtime=True, 
        save_intermediate_models=True, intermediate_model_prefix="intermediate_model_", save_training_logs=True);
ptf.Train();

Training params
    Num Epochs: 4

Display params
    Display progress:          True
    Display progress realtime: True
    Save Training logs:        True
    Save Intermediate models:  True
    Intermediate model prefix: intermediate_model_

Training Start
    Epoch 1/4
    ----------




    curr_lr - 0.001
    [Epoch 1] Train-acc: 0.570, Train-loss: 0.702 | Val-acc: 0.820000, Val-loss: 0.532, | time: 1.3 sec

    Epoch 2/4
    ----------




    curr_lr - 0.001
    [Epoch 2] Train-acc: 0.825, Train-loss: 0.471 | Val-acc: 0.880000, Val-loss: 0.396, | time: 1.4 sec

    Epoch 3/4
    ----------




    curr_lr - 0.001
    [Epoch 3] Train-acc: 0.830, Train-loss: 0.425 | Val-acc: 0.880000, Val-loss: 0.329, | time: 1.3 sec

    Epoch 4/4
    ----------




    curr_lr - 0.001
    [Epoch 4] Train-acc: 0.850, Train-loss: 0.386 | Val-acc: 0.960000, Val-loss: 0.265, | time: 1.5 sec

    Training completed in: 0m 4s
    Best val Acc:          0.960000

Training End

Training Outputs
    Model Dir:   /home/abhi/Desktop/Work/tess_tool/gui/v0.3/finetune_models/Resources/v0.1/workspace/exp-1/proj-1/output/models/
    Log Dir:     /home/abhi/Desktop/Work/tess_tool/gui/v0.3/finetune_models/Resources/v0.1/workspace/exp-1/proj-1/output/logs/
    Final model: final
    Best model:  best_model
    Log 1 - Validation accuracy history log: val_acc_history.npy
    Log 2 - Validation loss history log:     val_loss_history.npy
    Log 3 - Training accuracy history log:   train_acc_history.npy
    Log 4 - Training loss history log:       train_loss_history.npy
    Log 5 - Training curve:                  train_loss_history.npy
    Log 6 - Validation curve:                train_loss_history.npy



<Figure size 432x288 with 0 Axes>